In [1]:
# Package Install and Imports
from natsort import natsorted
import pandas as pd 
from geopy.geocoders import Nominatim
import time
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import time
import folium
import numpy as np
import plotly.graph_objects as go
from folium.plugins import MarkerCluster
import os

In [2]:

def clean_data(event_file):
    """
    Cleaning Function for Documents Formated Like the Pool Data Provided by Community Partner. 
    Removes unnecessary granularity in addresses, and concatenates different parts of address.
    Due to expansion of this program for additional activity files, there is less attention 
    to detail in solving individual formatting programs. 

    Parameters:
        - event_file Excel Filename 

    Returns:
        - pd.DataFrame: event_data, which is used throughout the notebook
    """
    event_data = pd.read_excel(event_file) 

    # Drop column: 'Event Name'
    event_data = event_data.drop(columns=['Event Name'])
    # Drop column: 'Address 2'
    event_data = event_data.drop(columns=['Address 2'])
    # Adding Leading 0 to Zip Code
    event_data['Zip Code'] = event_data['Zip Code'].astype(str).str.zfill(5) 
    # Change column type to string for column: 'Grade'
    event_data = event_data.astype({'Grade': 'string'})
    # Filling Empty Grade Observations with 13
    event_data['Grade'].fillna('N/A', inplace=True)
    # Convert text to uppercase in columns: 'City', 'State', 'Address'
    event_data['City'] = event_data['City'].str.upper()
    event_data['State'] = event_data['State'].str.upper()
    event_data['Address'] = event_data['Address'].str.upper()
    # Removing all addresses that include 1/2
    event_data['Address'] = event_data['Address'].str.replace(r'\s1/2', '', regex=True)
    # Creating a new column that will concatenate the address
    event_data['complete_add'] = event_data['Address'] + ', ' + event_data['City'] + ', ' + event_data['State'] + ', '+ event_data['Zip Code']
    return event_data

#event_data_clean = clean_data('/Users/carlosgonzalez/Desktop/Bates/DCS/Mock Data- Bball.xlsx')
#event_data_clean.head()

In [3]:
def geocode_addresses(df, user_agent):
    """
    Geocode addresses in a DataFrame and add latitude and longitude columns.
    Due to expansion of this program for additional activity files, there is less attention 
    to detail in solving individual formatting programs, addresses that could not be geocoded are instead dropped. 
    The returned DF of this function is also later used for maps and graphs. 


    Parameters:
        - df (pd.DataFrame): DataFrame containing addresses - this comes from clean_data(event_file):
        - user_agent (str): User agent for geocoding service . This should be used email
        according to the TOS of Nominatim if the query is going to be big. 

    Returns:
        - pd.DataFrame: DataFrame (event_data) with added 'latitude' and 'longitude' columns.
    """
    geolocator = Nominatim(user_agent=user_agent)

    latitudes = []
    longitudes = []
    errors = []

    for address in df['complete_add']:
        try:
            location = geolocator.geocode(address, timeout=10)
            if location is not None:
                latitudes.append(location.latitude)
                longitudes.append(location.longitude)
            else:
                latitudes.append(None)
                longitudes.append(None)
                errors.append(f"Unable to geocode address: '{address}'. No location found.")
        except Exception as e:
            print(f"An error occurred for address '{address}': {str(e)}")
            latitudes.append(None)
            longitudes.append(None)
            errors.append(f"Error geocoding address '{address}': {str(e)}") # errors not rlly used but saved just in case

    df['latitude'] = latitudes
    df['longitude'] = longitudes

    # Drop rows with missing latitude or longitude
    df = df.dropna(subset=['longitude', 'latitude'])

    return df # had troubled down the road if returned error and df tuple, so returning just df for now


In [4]:
def census_tract_map(event_name, event_coords, census_shapefile_path, demographic_data_path):
    """
    Function that creates DF with demographic information and census TRACT file
        - Information on demographic data sources and shapefiles available on read.me file

    Parameters:
        - event_name (str): name of the event for which the activity report is being done
        - event_coords (pd.DataFrame): DataFrame containing pool coordinates with 'latitude' and 'longitude'.
        - census_shapefile_path (str): Path to the census TRACT shapefile.
        - demographic_data_path (str): Specific demographic data filepath , available in GitHub

    Returns:
        - m (folium.Map) : Folium map center at Kennedy Pool, Lewiston, ME with census TRACT boundaries 
    """
    # Create the 'geom' column with Point geometries
    event_coords['geom'] = [Point(xy) for xy in zip(event_coords['longitude'], event_coords['latitude'])]
    # Create a GeoDataFrame for pool visitor locations with EPSG:4326 CRS
    event_data = gpd.GeoDataFrame(event_coords, geometry='geom', crs='EPSG:4326')

    # Read and match the census shapefile
    census = gpd.read_file(census_shapefile_path)
    census = census.to_crs('EPSG:4326')  # Match with coordinates format

    # Perform the spatial join based on 'within'
    # Left Join - will match coordinate observations with the census tract they are 'within'
    event_data = gpd.sjoin(event_data, census, how="left", predicate="within")
    event_data['GEOID'] = event_data['GEOID'].fillna(0).astype('int64')
    event_data['GEOID'] = event_data['GEOID'].astype('int64')
    census['GEOID'] = census['GEOID'].astype('int64')   # There was some mismatch in the types, addressing now for future join


    # Creating a df that is just the GEOID and the registration counts
    event_counts = event_data.groupby('GEOID').size().reset_index(name='registration_count')
    event_counts = census.merge(event_counts, how='right')  #including the census shapefile AGAIN

    dems = pd.read_csv(demographic_data_path) 
    event_counts_dems = pd.merge(event_counts, dems, left_on='GEOID', right_on='Geo_FIPS', how='left')  # Adding demographic info
    event_counts_dems = event_counts_dems[event_counts_dems['geometry'].notnull() & (event_counts_dems['geometry'] != '')]  # Dropping obs that do not have geom 
    event_counts_dems.fillna(-99, inplace=True) # Filling in any missing demographic info with -99

    # Creatng Map
    m = folium.Map(location=[44.094031, -70.214110], zoom_start=14)

    title_html = f'''
                 <h3 style="text-align: center; font-size: 24px; padding: 10px; background-color: #f0f0f0; border-radius: 8px; border: 2px solid #ccc;"><b>{event_name} Registered Members</b></h3>
                 ''' # Styling HTML from Stackoverflow :)
    m.get_root().html.add_child(folium.Element(title_html))


    choropleth = folium.Choropleth(
        geo_data=event_counts_dems,
        data=event_counts_dems,
        columns=['GEOID', 'registration_count'],
        key_on='feature.properties.GEOID',
        fill_color='YlGnBu',
        legend_name='Number of Registered Event Participants',
        highlight=True,
    ).add_to(m)

    for idx, row in event_counts_dems.iterrows():
        block_name = row['NAMELSAD']
        visitors = row['registration_count']
        unemployed = round(row['SE_T067_006'])
        dropped = round(row['SE_T057_002'])
        poverty = round(row['SE_T090_002'])

        popup_content = f"{block_name}<br>Visitors: {visitors}<br>Total Number of Unemployed Residents: {unemployed}<br>Students 16-19 Not High School Graduate, not Enrolled (Dropped Out): {dropped}<br>Number of houses with income <$10,000.00: {poverty}"

        folium.GeoJson(
            row['geometry'],
            name='geojson',
            style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
            highlight_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
            popup=folium.Popup(popup_content, max_width=300),
        ).add_to(m)

    return m



In [5]:
def census_blockgroup_map(event_name, event_coords, census_blockg_shapefile_path):
    """
    Function that creates DF with demographic information and census block group file

    Parameters:
        - event_name (str): name of the event for which the activity report is being done
        - event_data (pd.DataFrame): DataFrame containing event coordinates with 'latitude' and 'longitude'.
        - census_blockg_shapefile_path (str): Path to the census BLOCK GROUP shapefile.

    Returns:
        - m (folium.Map) : Folium map center at Kennedy Pool, Lewiston, ME with census block groups boundaries 
    """
    # Create the 'geom' column with Point geometries
    event_coords['geom'] = [Point(xy) for xy in zip(event_coords['longitude'], event_coords['latitude'])]
    # Create a GeoDataFrame for pool visitor locations with EPSG:4326 CRS
    event_data = gpd.GeoDataFrame(event_coords, geometry='geom', crs='EPSG:4326')

    # Read and match the census shapefile
    census = gpd.read_file(census_blockg_shapefile_path)
    census = census.to_crs('EPSG:4326')  # Match with coordinates format

    # Perform the spatial join based on 'within'
    # Left Join - will match coordinate observations with the census tract they are 'within'
    event_data = gpd.sjoin(event_data, census, how="left", predicate="within")

    event_data = event_data.groupby('GEOID20').size().reset_index(name='registration_count')
    # Merge pool counts with the census gdf
    # Doing a right join so we only keep census tracts with pools
    event_data = census.merge(event_data, how='right')

    event_data['visitors_per_1000_residents'] = (event_data['registration_count'] / event_data['POP20']).replace(np.inf, 0) * 1000
    event_data['visitors_per_1000_residents'] = event_data['visitors_per_1000_residents'].round(2)


    # Creatng Map
    m = folium.Map(location=[44.094031, -70.214110], zoom_start=14)  # Specify the map center and initial zoom level

    title_html = f'''
                 <h3 style="text-align: center; font-size: 24px; padding: 10px; background-color: #f0f0f0; border-radius: 8px; border: 2px solid #ccc;"><b>{event_name} Registered Members</b></h3>
                 ''' # Styling HTML from Stackoverflow :)
    m.get_root().html.add_child(folium.Element(title_html))


    choropleth = folium.Choropleth(
        geo_data=event_data,
        data=event_data,
        columns=['GEOID20', 'visitors_per_1000_residents'],
        key_on='feature.properties.GEOID20',
        fill_color='YlGnBu',
        legend_name='Number of Visitors per 1000 Residents',
        highlight=True,
    ).add_to(m)

    # Add popups for census blocks with information
    for idx, row in event_data.iterrows():
        block_name = row['BLOCKCE20']
        visitors = row['registration_count']
        visitors_per_1000 = row['visitors_per_1000_residents']
        
        popup_content = f"Block Name: {block_name}<br>Visitors: {visitors}<br>Visitors per 1000: {visitors_per_1000}"

        folium.GeoJson(
            row['geometry'],
            name='geojson',
            style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
            highlight_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
            popup=folium.Popup(popup_content, max_width=300),
        ).add_to(m)

        return m


In [6]:
def demographs_graphs(event_data):
    """
    Function that 2 graphs showing gender and grade distribution for the visitors 

    Parameters:
        - event_data (pd.DataFrame): DataFrame containing event coordinates with 'latitude' and 'longitude'.

    Returns:
        - fig, fig1 (go.Figure): Tuple containing graphs for gender and grade
    """

    event_data['Grade'].fillna('No Grade', inplace=True) # Cases for NaNs
    event_data['Gender'].fillna('Gender Not Disclosed', inplace=True) # Cases for NaNs

    # Group by 'Grade' and calculate the count
    event_grades = event_data.groupby('Grade').size().reset_index(name='grade_count')
    event_grades['Grade'] = natsorted(event_grades['Grade'])

    # Group by 'Gender' and calculate the count
    event_gender = event_data.groupby('Gender').size().reset_index(name='gender_count')

    # Creating Gender and Grade figures
    fig = go.Figure(data=[go.Pie(labels=event_gender['Gender'], values=event_gender['gender_count'])])
    fig.update_layout(title_text='Activity Visit Distribution By Gender')

    fig.write_html("gender_visits.html")

    fig1 = go.Figure(data=[go.Bar(x=event_grades['Grade'], y=event_grades['grade_count'])])
    fig1.update_layout(title_text='Activity Visit Distribution By Grade',
                       xaxis_title='Grade',
                       yaxis_title='Visits')

    return fig, fig1  # Returning both fig and fig1 as a tuple


In [7]:

def activityReport(event_name, event_file, user_agent, census_shapefile_path, demographic_data_path, census_blockg_shapefile_path):
    """
    Function that runs alls the functions above, and generates complete activity report 

    Parameters:
        - event_name (str): name of the event for which the activity report is being done
        - event_file (str): Excel activity file path
        - user_agent (str): User agent for geocoding service .
        - census_shapefile_path (str): Path to the census BLOCK GROUP shapefile.
        - demographic_data_path (str): Specific demographic data filepath , available in GitHub
        - census_blockg_shapefile_path (str): Path to the census BLOCK GROUP shapefile.

    Returns:
        - directory "event_name" which contains: census_tract_map, census_blockgroup_map, gender_visits.html, grade_visits.html
    """

    # Creating directory with event_name if it doesn't exist
    if not os.path.exists(event_name):
        os.makedirs(event_name)

    event_data_clean = clean_data(event_file)
    print('clean_data complete')
    event_data = geocode_addresses(event_data_clean, user_agent)
    print('geocode_addresses complete')

    # Saving maps and graphs within the event_name directory
    os.chdir(event_name)  # Changing working directory to the event_name directory

    map1 = census_tract_map(event_name, event_data, census_shapefile_path, demographic_data_path)
    print('census_tract_map complete')
    map1.save('census_tract_map.html')  # Saving map1 inside the event_name directory

    map2 = census_blockgroup_map(event_name, event_data, census_blockg_shapefile_path)
    print('census_blockgroup_map complete')
    map2.save('census_blockgroup_map.html')  # Saving map2 inside the event_name directory

    dem = demographs_graphs(event_data)
    fig, fig1 = dem 
    print('demographs_graphs complete')
    fig.write_html("gender_visits.html")
    fig1.write_html("grade_visits.html")

    os.chdir('..')  # Changing back to the parent directory

    print('Activity Report Complete')    
    

In [8]:
activityReport("City of Lewiston Basketball Club - Mock Data", '/Users/carlosgonzalez/Desktop/Bates/DCS/mock_basketball_data.xlsx', "cgonzal2@bates.edu", 
               '/Users/carlosgonzalez/Desktop/Bates/DCS/maine_shp/tl_2020_23_tract.shp', '/Users/carlosgonzalez/Desktop/Bates/DCS/dems.csv', 
               "/Users/carlosgonzalez/Desktop/Bates/DCS/maine_blocks/tl_2020_23_tabblock20.shp")

activityReport("Kennedy Pool", '/Users/carlosgonzalez/Desktop/Bates/DCS/pool_data.xlsx', "cgonzal2@bates.edu", 
               '/Users/carlosgonzalez/Desktop/Bates/DCS/maine_shp/tl_2020_23_tract.shp', '/Users/carlosgonzalez/Desktop/Bates/DCS/dems.csv', 
               "/Users/carlosgonzalez/Desktop/Bates/DCS/maine_blocks/tl_2020_23_tabblock20.shp")

activityReport("City of Lewiston Chess Club - Mock Data", '/Users/carlosgonzalez/Desktop/Bates/DCS/mock_chess_data.xlsx', "cgonzal2@bates.edu", 
               '/Users/carlosgonzalez/Desktop/Bates/DCS/maine_shp/tl_2020_23_tract.shp', '/Users/carlosgonzalez/Desktop/Bates/DCS/dems.csv', 
               "/Users/carlosgonzalez/Desktop/Bates/DCS/maine_blocks/tl_2020_23_tabblock20.shp")

clean_data complete
geocode_addresses complete
census_tract_map complete
census_blockgroup_map complete
demographs_graphs complete
Activity Report Complete
clean_data complete
geocode_addresses complete


/var/folders/rj/d5bkq5lj19xfb9d5g8z1jk700000gn/T/ipykernel_6978/1760121470.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



census_tract_map complete


/var/folders/rj/d5bkq5lj19xfb9d5g8z1jk700000gn/T/ipykernel_6978/320890636.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



census_blockgroup_map complete
demographs_graphs complete
Activity Report Complete
clean_data complete


/var/folders/rj/d5bkq5lj19xfb9d5g8z1jk700000gn/T/ipykernel_6978/854355597.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rj/d5bkq5lj19xfb9d5g8z1jk700000gn/T/ipykernel_6978/854355597.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



geocode_addresses complete
census_tract_map complete
census_blockgroup_map complete
demographs_graphs complete
Activity Report Complete
